Autor: mzibecchi  
email: mzibecchi@gmail.com  
  
_21-Sep: Reglas de asociacion por genero agregadas_

In [2]:
## Aprendizaje de reglas de asociación

Objetivo:
    
    Obtener reglas de asociación entre películas en el dataset movielens.
    Asumimos que los usuarios hacen recomendaciones y queremos encontras reglas de asociacion entre esas recomendaciones.
    Tomamos como premisa que el usuario recomienda peliculas que superan cierto rating.
     

In [3]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

Descripcion de los datos:

movies.csv (movieId, title, genres)
ratings.csv (userId, movieId, rating, timestamp)



Hacemos el desarrollo sobre un dataset mas chico, luego pasamos al de 20m

In [4]:
# small dataset
path = "ml-latest-small"

# 20m records dataset
#path = "ml-20m"

In [5]:
#este dataset finalmente no lo usamos
tags_df = pd.read_csv(path+"/tags.csv")
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


Exploramos que hay en tags, para ver si podemos usar el tag como una categoria que luego pueda generar alguna asociacion...

In [6]:
tags_df.tag.unique()

array(['funny', 'Highly quotable', 'will ferrell', ..., 'gun fu',
       'heroic bloodshed', 'Heroic Bloodshed'], dtype=object)

In [7]:
len(tags_df.tag.unique())

1589

In [8]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [9]:
movies_df = pd.read_csv(path+"/movies.csv")
print(len(movies_df))
movies_df.head()

9742


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Revisemos genres, para ver que alternativas de clasificacion tenemos aca...

In [10]:
movies_df.genres.unique()

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance',
       'Comedy|Drama|Romance', 'Comedy', 'Action|Crime|Thriller',
       'Adventure|Children', 'Action', 'Action|Adventure|Thriller',
       'Comedy|Horror', 'Adventure|Animation|Children', 'Drama',
       'Action|Adventure|Romance', 'Crime|Drama', 'Drama|Romance',
       'Action|Comedy|Crime|Drama|Thriller', 'Comedy|Crime|Thriller',
       'Crime|Drama|Horror|Mystery|Thriller', 'Drama|Sci-Fi',
       'Children|Drama', 'Adventure|Drama|Fantasy|Mystery|Sci-Fi',
       'Mystery|Sci-Fi|Thriller', 'Children|Comedy', 'Drama|War',
       'Action|Crime|Drama', 'Action|Adventure|Fantasy',
       'Comedy|Drama|Thriller', 'Mystery|Thriller',
       'Animation|Children|Drama|Musical|Romance',
       'Crime|Mystery|Thriller', 'Adventure|Drama', 'Drama|Thriller',
       'Comedy|Crime', 'Action|Sci-Fi|Thriller',
       'Action|Comedy|Horror|Thriller', 'Comedy|Drama', 'Documentary',
       'Ac

In [11]:
movies_df.groupby('genres').count()

,movieId,title
genres,,
(no genres listed),34,34
Action,60,60
Action|Adventure,32,32
Action|Adventure|Animation,18,18
Action|Adventure|Animation|Children,6,6
Action|Adventure|Animation|Children|Comedy,5,5
Action|Adventure|Animation|Children|Comedy|Fantasy,3,3
Action|Adventure|Animation|Children|Comedy|IMAX,2,2
Action|Adventure|Animation|Children|Comedy|Romance,1,1


In [12]:
my_genres_list = ['Western', 
                  'Thriller',
                 'Sci-Fi',
                 'Romance',
                 'Mystery',
                 'Musical',
                 'Horror',
                 'War',
                 'Comedy',
                 'Children']

si genres tiene un solo elemento - esa es el genero  
si genres tiene mas de un elemento:  
    convierto genres a una lista  
    si en la lista, hay algun elemento de la lista de genres definidas por mi, tomo ese elemento
    si en la lista no hay ninguno, pongo ese elemento

In [29]:
def genres_choose( genreslist ):
    # si hay mas de un genero, nos quedamos con uno de los que figuran en mi lista de generos
    
    coincidencias = [x for x in genreslist if x in my_genres_list]
    
    if coincidencias == []:
        return genreslist[0]
    
    return coincidencias[0]
    

def genres_unificar( genres ):
    # transformamos la lista de generos en un solo genero
    
    genlist = genres.split('|')
    
    if len(genlist) == 1:
        return genlist[0]
    else:
        return genres_choose(genlist)

In [14]:
selector_genero = lambda x: genres_unificar(x)

movies_df.genres = movies_df.genres.apply ( selector_genero )

In [15]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Children
1,2,Jumanji (1995),Children
2,3,Grumpier Old Men (1995),Comedy
3,4,Waiting to Exhale (1995),Comedy
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
ratings1_df = pd.read_csv(path+"/ratings.csv")
print(len(ratings1_df))
ratings1_df.head()

100836


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Hacemos merge de ambos datasets, usando movieId como clave para el join

In [17]:
ratings_df = pd.merge(ratings1_df, movies_df, on='movieId')
ratings_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Children
1,5,1,4.0,847434962,Toy Story (1995),Children
2,7,1,4.5,1106635946,Toy Story (1995),Children
3,15,1,2.5,1510577970,Toy Story (1995),Children
4,17,1,4.5,1305696483,Toy Story (1995),Children


Veamos que tipos de ratings existen

In [18]:
ratings_df.rating.unique()

array([4. , 4.5, 2.5, 3.5, 3. , 5. , 0.5, 2. , 1.5, 1. ])

Asumiremos que una pelicula recomendada es aquella que tiene rating > 2

In [19]:
ratings_df["recommended"] = ratings_df["rating"] > 2
recommended_df = ratings_df[ ratings_df.recommended == True]
recommended_df.head()

,userId,movieId,rating,timestamp,title,genres,recommended
0,1,1,4.0,964982703,Toy Story (1995),Children,True
1,5,1,4.0,847434962,Toy Story (1995),Children,True
2,7,1,4.5,1106635946,Toy Story (1995),Children,True
3,15,1,2.5,1510577970,Toy Story (1995),Children,True
4,17,1,4.5,1305696483,Toy Story (1995),Children,True


In [20]:
# la version 2.0 de esta notebook, va a tener una implementacion mas eficiente de esto...
# por ahora, me interesaba mas entender como armar las transacciones

transactions = []

grouped = recommended_df.groupby('userId')

for names, group  in grouped:
    
    transaction_user = []
    
    for row_index, movie in group.iterrows():
        transaction_user.append( movie['title'] )
        
    transactions.append( transaction_user )
    

In [21]:
#transactions

In [22]:
print("Cantidad de transacciones: "+str(len(transactions)))

Cantidad de transacciones: 610


In [23]:
from efficient_apriori import apriori

#apriori de efficient_apriori requiere una lista de transacciones (no necesita que estén ordenados en cada transacción)

itemsets, rules = apriori(transactions, min_support=0.3,  min_confidence=0.2, max_length=5)#min_sup conf entre 0 y 1

print("\nItemsets:\n")
print(itemsets)

rules=sorted(rules, key=lambda rule: rule.confidence)

print("\nRules:\n")

for rule in rules:
    
  print(rule) # Prints the rule and its confidence, support, lift, ...

  print("\n")


Itemsets:

{1: {('American Beauty (1999)',): 192, ('Apollo 13 (1995)',): 188, ('Braveheart (1995)',): 224, ('Fight Club (1999)',): 209, ('Forrest Gump (1994)',): 318, ('Fugitive, The (1993)',): 186, ('Jurassic Park (1993)',): 221, ('Lord of the Rings: The Fellowship of the Ring, The (2001)',): 185, ('Matrix, The (1999)',): 264, ('Pulp Fiction (1994)',): 290, ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',): 195, ('Saving Private Ryan (1998)',): 183, ("Schindler's List (1993)",): 208, ('Seven (a.k.a. Se7en) (1995)',): 191, ('Shawshank Redemption, The (1994)',): 313, ('Silence of the Lambs, The (1991)',): 270, ('Star Wars: Episode IV - A New Hope (1977)',): 239, ('Star Wars: Episode V - The Empire Strikes Back (1980)',): 204, ('Star Wars: Episode VI - Return of the Jedi (1983)',): 186, ('Terminator 2: Judgment Day (1991)',): 216, ('Toy Story (1995)',): 207, ('Usual Suspects, The (1995)',): 199}, 2: {('Forrest Gump (1994)', 'Pulp Fiction (1994)'): 211, 

In [24]:
print(len(rules))

12


## Reglas de Asociacion usando el genero

In [25]:
# la version 2.0 de esta notebook, va a tener una implementacion mas eficiente de esto...
# por ahora, me interesaba mas entender como armar las transacciones

transactions = []

grouped = recommended_df.groupby('userId')

for names, group  in grouped:
    
    transaction_user = []
    
    for row_index, movie in group.iterrows():
        transaction_user.append( movie['genres'] )
        
    transactions.append( transaction_user )
    

In [26]:
len(transactions)

610

In [28]:
from efficient_apriori import apriori

#apriori de efficient_apriori requiere una lista de transacciones (no necesita que estén ordenados en cada transacción)

itemsets, rules = apriori(transactions, min_support=0.8,  min_confidence=0.6, max_length=5)#min_sup conf entre 0 y 1

#print("\nItemsets:\n")
#print(itemsets)

rules=sorted(rules, key=lambda rule: rule.confidence)

print("\nRules:\n")

for rule in rules:
    
  print(rule) # Prints the rule and its confidence, support, lift, ...

  print("\n")


Rules:

{Comedy} -> {Horror} (conf: 0.803, supp: 0.800, lift: 1.001, conv: 1.005)


{Comedy} -> {Mystery, Romance, Thriller} (conf: 0.803, supp: 0.800, lift: 1.003, conv: 1.013)


{Comedy} -> {Romance, Sci-Fi, Thriller} (conf: 0.804, supp: 0.802, lift: 1.003, conv: 1.013)


{Comedy} -> {Children, Mystery} (conf: 0.812, supp: 0.810, lift: 1.003, conv: 1.014)


{Comedy} -> {Children, Sci-Fi} (conf: 0.824, supp: 0.821, lift: 1.003, conv: 1.015)


{Comedy} -> {Children, Thriller} (conf: 0.827, supp: 0.825, lift: 1.003, conv: 1.016)


{Comedy} -> {Mystery, Romance} (conf: 0.827, supp: 0.825, lift: 1.003, conv: 1.016)


{Comedy} -> {Drama} (conf: 0.832, supp: 0.830, lift: 1.003, conv: 1.016)


{Comedy} -> {Mystery, Sci-Fi, Thriller} (conf: 0.832, supp: 0.830, lift: 1.001, conv: 1.006)


{Comedy} -> {Romance, Sci-Fi} (conf: 0.836, supp: 0.833, lift: 1.003, conv: 1.017)


{Comedy} -> {Romance, Thriller} (conf: 0.844, supp: 0.841, lift: 1.003, conv: 1.018)


{Thriller} -> {Mystery, Romance} (c